In [1]:
import pandas as pd
import numpy as np
import requests
import os
from dotenv import load_dotenv
import re

In [2]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [3]:
# busan_restrua_2019_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_201912.csv"
busan_restrua_2020_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_20200720.csv"
busan_restrua_2021_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_202106.csv"
busan_restrua_2022_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_2022.csv"
busan_restrua_2023_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/BC_GUGUN_PRGN_RSTRT_STTUS_2023.csv"

# restaurants_19 = pd.read_csv(busan_restrua_2019_dir)
restaurants_20 = pd.read_csv(busan_restrua_2020_dir)
restaurants_21 = pd.read_csv(busan_restrua_2021_dir)
restaurants_22 = pd.read_csv(busan_restrua_2022_dir)
restaurants_23 = pd.read_csv(busan_restrua_2023_dir)

In [4]:
df_list = [restaurants_20, restaurants_21, restaurants_22, restaurants_23]

# 데이터프레임 합치기
busan_restrau_df = pd.concat(df_list)

# 'INDUTY_NM' 컬럼을 기준으로 중복 제거
busan_restrau_df = busan_restrau_df.drop_duplicates(subset='RSTRNT_NM').reset_index(drop=True)

In [5]:
busan_restrau_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SYS_NO           627 non-null    int64  
 1   INDUTY_NM        627 non-null    object 
 2   BZCND_NM         627 non-null    object 
 3   RSTRNT_NM        626 non-null    object 
 4   LOCPLC_ALL_ADDR  626 non-null    object 
 5   LOCPLC_LNM_ADDR  626 non-null    object 
 6   MENU_NM          626 non-null    object 
 7   TEL_NO           626 non-null    object 
 8   APPN_DE          626 non-null    object 
 9   RSNMT_DE         626 non-null    object 
 10  GUGUN_NM         626 non-null    object 
 11  DATA_BASE_DE     626 non-null    object 
 12  GID_LA           626 non-null    float64
 13  GID_LO           626 non-null    float64
dtypes: float64(2), int64(1), object(11)
memory usage: 68.7+ KB


In [9]:
df_rating_missing = busan_restrau_df[busan_restrau_df['LOCPLC_LNM_ADDR'].isnull()]
df_rating_missing

,SYS_NO,INDUTY_NM,BZCND_NM,RSTRNT_NM,LOCPLC_ALL_ADDR,LOCPLC_LNM_ADDR,MENU_NM,TEL_NO,APPN_DE,RSNMT_DE,GUGUN_NM,DATA_BASE_DE,GID_LA,GID_LO
614,227,일반음식점,외국음식전문점(인도,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
busan_restrau_df = busan_restrau_df[~((busan_restrau_df['SYS_NO'] == 227) & (busan_restrau_df['BZCND_NM'] == '외국음식전문점(인도'))].reset_index(drop=True)

In [11]:
busan_restrau_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SYS_NO           626 non-null    int64  
 1   INDUTY_NM        626 non-null    object 
 2   BZCND_NM         626 non-null    object 
 3   RSTRNT_NM        626 non-null    object 
 4   LOCPLC_ALL_ADDR  626 non-null    object 
 5   LOCPLC_LNM_ADDR  626 non-null    object 
 6   MENU_NM          626 non-null    object 
 7   TEL_NO           626 non-null    object 
 8   APPN_DE          626 non-null    object 
 9   RSNMT_DE         626 non-null    object 
 10  GUGUN_NM         626 non-null    object 
 11  DATA_BASE_DE     626 non-null    object 
 12  GID_LA           626 non-null    float64
 13  GID_LO           626 non-null    float64
dtypes: float64(2), int64(1), object(11)
memory usage: 68.6+ KB


In [14]:
# busan_restrau_df = busan_restrau_df.drop(columns=["SYS_NO"])
busan_restrau_df.insert(0, 'SYS_NO', busan_restrau_df.index)

In [15]:
busan_restrau_df.head()

,SYS_NO,INDUTY_NM,BZCND_NM,RSTRNT_NM,LOCPLC_ALL_ADDR,LOCPLC_LNM_ADDR,MENU_NM,TEL_NO,APPN_DE,RSNMT_DE,GUGUN_NM,DATA_BASE_DE,GID_LA,GID_LO
0,0,일반음식점,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),부산광역시 연제구 연산동 1241번지 9호,돼지갈비,051-852-0218,2014-10-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.187,129.080
1,1,일반음식점,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",부산광역시 연제구 연산동 1127번지 5호 1층,대구탕,051-867-7611,2011-11-09 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.187,129.080
2,2,일반음식점,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)","부산광역시 연제구 연산동 1242번지 8호 ,13",뷔페,051-867-7600,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.186,129.081
3,3,일반음식점,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),부산광역시 연제구 연산동 1242번지 6호,보쌈,051-853-8005,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.186,129.080
4,4,일반음식점,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),부산광역시 연제구 거제동 1490번지 2호,중식,051-507-9555,2010-06-30 12:00,2018-10-26 12:00,연제구,2019-05-07 12:00,35.191,129.075


In [16]:
busan_restrau_df.tail()

,SYS_NO,INDUTY_NM,BZCND_NM,RSTRNT_NM,LOCPLC_ALL_ADDR,LOCPLC_LNM_ADDR,MENU_NM,TEL_NO,APPN_DE,RSNMT_DE,GUGUN_NM,DATA_BASE_DE,GID_LA,GID_LO
621,621,일반음식점,복어취급,금수복국 온천점,부산광역시 동래구 온천장로107번길 18 (온천동),부산광역시 동래구 온천동 137-12,복국,051-555-9912,2007-06-13,2023-10-30,부산광역시 동래구,2023-12-31,35.220544,129.083349
622,622,일반음식점,한식,쌍둥이돼지국밥 본점,부산광역시 남구 유엔평화로 35-1 (대연동),부산광역시 남구 대연동 889-5,돼지국밥,051-628-7021,2016-11-08,2023-10-31,부산광역시 남구,2024-01-08,35.131770,129.092306
623,623,일반음식점,한식,오륙도 가원,부산광역시 남구 백운포로 14 (용호동),부산광역시 남구 용호동 894-54,소 숯불구이,051-635-0707,2014-10-31,2023-10-31,부산광역시 남구,2024-01-08,35.106842,129.111358
624,624,일반음식점,중국,가보자짬뽕,부산광역시 기장군 장안읍 해맞이로 425,부산광역시 기장군 장안읍 길천리 205-4,중국요리,051-727-3911,2008-06-16,2023-12-21,부산광역시 기장군,2024-01-09,35.329756,129.281918
625,625,일반음식점,한식,금정어가,부산광역시 두실로 17(구서동),부산광역시 금정구 구서동 159-45,장어구이,051-514-8216,2001-07-02,2023-10-31,부산광역시 금정구,2023-12-31,35.260477,129.089353


In [17]:
# busan_restrau_df.to_csv("./datas/busan_restrau_20to24.csv", index=False, encoding="utf-8")

In [18]:
busan_restrau_format = busan_restrau_df[['BZCND_NM', 'RSTRNT_NM', 'LOCPLC_ALL_ADDR', 'MENU_NM', 'TEL_NO', 'GUGUN_NM', 'GID_LA', 'GID_LO']].copy()
busan_restrau_format.rename(columns={'BZCND_NM':'MENU_TYPE', 'RSTRNT_NM':'RESTAURANT_NAME_KOREAN', 'LOCPLC_ALL_ADDR': 'ADDRESS_KOREAN', 'MENU_NM':'MENU_NAME', 'TEL_NO':'NATIONAL_PHONE_NUMBER', 'GUGUN_NM':'DISTRICT_NAME', 'GID_LA':'LATITUDE', 'GID_LO':'LONGITUDE'},inplace = True)
busan_restrau_format.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE
0,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),돼지갈비,051-852-0218,연제구,35.187,129.080
1,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",대구탕,051-867-7611,연제구,35.187,129.080
2,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)",뷔페,051-867-7600,연제구,35.186,129.081
3,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),보쌈,051-853-8005,연제구,35.186,129.080
4,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075


In [3]:
def get_place_id(restaurant_name,location):
    search_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    search_params = {
        'input': restaurant_name,
        'inputtype': 'textquery',
        'fields': 'place_id',
        'key': GOOGLE_API_KEY,
        'locationbias': f'circle:2000@{location}'
    }
    response = requests.get(search_url, params=search_params)
    results = response.json().get('candidates', [])
    print(response.json())
    if results:
        return results[0]['place_id']
    else:
        return None
    
# Function to fetch detailed information using the place ID
def get_place_details_new(place_id):
    details_url = f"https://places.googleapis.com/v1/places/{place_id}"
    details_params = {
        'fields': 'displayName,types,formattedAddress,priceLevel,rating,userRatingCount,regularOpeningHours,accessibilityOptions,goodForChildren,goodForGroups,menuForChildren,parkingOptions,paymentOptions,outdoorSeating,reservable,restroom,servesBeer,servesBreakfast,servesBrunch,servesDinner,servesLunch,servesVegetarianFood,takeout,reviews',
        'key': GOOGLE_API_KEY
    }
    response = requests.get(details_url, params=details_params)
    return response.json()

In [4]:
def format_review(response):
    if not response: return None
    total_review = ''
    for review in response:
        total_review += review.get('text').get('text')  + '\n' if review.get('text') else ''
    return total_review

def format_multiple_options(response):
    if not response: return None
    true_options = [k for k, v in response.items() if v]
    return ', '.join(true_options)

# def format_opening_hours(response):
#     if response.get('regularOpeningHours'): return None
#     return '\n'.join([re.sub(r'[^\x00-\x7F–]', '', text) for text in response.get('regularOpeningHours').get('weekdayDescriptions')])

def format_opening_hours(response):
    if not response.get('regularOpeningHours'): 
        print("None!!!")
        return None
    return '\n'.join([re.sub(r'[^\x00-\x7F–]', '', text) for text in response.get('regularOpeningHours').get('weekdayDescriptions')])



In [5]:
# 35.228689,129.091801
restaurant_name = "일송면옥"
location = "35.228689,129.091801" 

# Step 1: Get Place ID for the restaurant
place_id = get_place_id(restaurant_name, location)

if place_id:
    details = get_place_details_new(place_id)
    
    for key, value in details.items():
        if key != 'kreviews':
            print(f"{key}; {value}")
    
    # Print the reviews (optional)
    print("\nReviews:")
    print(details.get("reviews"))
    formatted_review = format_review(details.get("reviews"))
    print(formatted_review)

else:
    print("Restaurant not found.")

{'candidates': [{'place_id': 'ChIJ6YgQgeqTaDUR7gMrB95VEUw'}], 'status': 'OK'}
types; ['korean_restaurant', 'restaurant', 'food', 'point_of_interest', 'establishment']
formattedAddress; South Korea, Busan, Geumjeong-gu, 부곡2동 273-25
rating; 4.1
regularOpeningHours; {'openNow': False, 'periods': [{'open': {'day': 0, 'hour': 11, 'minute': 0}, 'close': {'day': 0, 'hour': 22, 'minute': 0}}, {'open': {'day': 1, 'hour': 11, 'minute': 0}, 'close': {'day': 1, 'hour': 22, 'minute': 0}}, {'open': {'day': 2, 'hour': 11, 'minute': 0}, 'close': {'day': 2, 'hour': 22, 'minute': 0}}, {'open': {'day': 3, 'hour': 11, 'minute': 0}, 'close': {'day': 3, 'hour': 22, 'minute': 0}}, {'open': {'day': 4, 'hour': 11, 'minute': 0}, 'close': {'day': 4, 'hour': 22, 'minute': 0}}, {'open': {'day': 5, 'hour': 11, 'minute': 0}, 'close': {'day': 5, 'hour': 22, 'minute': 0}}, {'open': {'day': 6, 'hour': 11, 'minute': 0}, 'close': {'day': 6, 'hour': 22, 'minute': 0}}], 'weekdayDescriptions': ['Monday: 11:00\u202fAM\u2009–

In [6]:
details

{'types': ['korean_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'formattedAddress': 'South Korea, Busan, Geumjeong-gu, 부곡2동 273-25',
 'rating': 4.1,
 'regularOpeningHours': {'openNow': False,
  'periods': [{'open': {'day': 0, 'hour': 11, 'minute': 0},
    'close': {'day': 0, 'hour': 22, 'minute': 0}},
   {'open': {'day': 1, 'hour': 11, 'minute': 0},
    'close': {'day': 1, 'hour': 22, 'minute': 0}},
   {'open': {'day': 2, 'hour': 11, 'minute': 0},
    'close': {'day': 2, 'hour': 22, 'minute': 0}},
   {'open': {'day': 3, 'hour': 11, 'minute': 0},
    'close': {'day': 3, 'hour': 22, 'minute': 0}},
   {'open': {'day': 4, 'hour': 11, 'minute': 0},
    'close': {'day': 4, 'hour': 22, 'minute': 0}},
   {'open': {'day': 5, 'hour': 11, 'minute': 0},
    'close': {'day': 5, 'hour': 22, 'minute': 0}},
   {'open': {'day': 6, 'hour': 11, 'minute': 0},
    'close': {'day': 6, 'hour': 22, 'minute': 0}}],
  'weekdayDescriptions': ['Monday: 11:00\u202fAM\u2009–\u2

In [31]:
test = format_opening_hours(details)

In [36]:
details.get("regularOpeningHours")

{'openNow': True,
 'periods': [{'open': {'day': 0, 'hour': 11, 'minute': 30},
   'close': {'day': 0, 'hour': 15, 'minute': 30}},
  {'open': {'day': 0, 'hour': 16, 'minute': 30},
   'close': {'day': 0, 'hour': 21, 'minute': 30}},
  {'open': {'day': 2, 'hour': 11, 'minute': 30},
   'close': {'day': 2, 'hour': 15, 'minute': 30}},
  {'open': {'day': 2, 'hour': 16, 'minute': 30},
   'close': {'day': 2, 'hour': 21, 'minute': 30}},
  {'open': {'day': 3, 'hour': 11, 'minute': 30},
   'close': {'day': 3, 'hour': 15, 'minute': 30}},
  {'open': {'day': 3, 'hour': 16, 'minute': 30},
   'close': {'day': 3, 'hour': 21, 'minute': 30}},
  {'open': {'day': 4, 'hour': 11, 'minute': 30},
   'close': {'day': 4, 'hour': 15, 'minute': 30}},
  {'open': {'day': 4, 'hour': 16, 'minute': 30},
   'close': {'day': 4, 'hour': 21, 'minute': 30}},
  {'open': {'day': 5, 'hour': 11, 'minute': 30},
   'close': {'day': 5, 'hour': 15, 'minute': 30}},
  {'open': {'day': 5, 'hour': 16, 'minute': 30},
   'close': {'day': 5,

In [35]:
type(details.get("regularOpeningHours"))

dict

In [22]:
import time

In [45]:
for i, row in busan_restrau_format.iterrows():
    time.sleep(0.1)
    restaurant_name = row["RESTAURANT_NAME_KOREAN"]
    location = f"{row['LATITUDE'],row['LONGITUDE']}" 
    place_id = get_place_id(restaurant_name, location)

    if place_id:
        details = get_place_details_new(place_id)
        print(details)

        busan_restrau_format.loc[i, 'RESTAURANT_NAME_ENGLISH'] = details.get('displayName').get('text') if details.get('displayName') and details.get('displayName').get('languageCode')=='en' else None
        busan_restrau_format.loc[i, 'ADDRESS_ENGLISH'] = details.get('formattedAddress')
        busan_restrau_format.loc[i, 'OPENING_HOURS'] = format_opening_hours(details)
        busan_restrau_format.loc[i, 'RATING'] = details.get('rating')
        busan_restrau_format.loc[i, 'RATING_COUNT'] = details.get('userRatingCount')
        busan_restrau_format.loc[i, 'PRICE_LEVEL'] = details.get('priceLevel')
        busan_restrau_format.loc[i, 'TAKEOUT_YN'] = details.get('takeout')
        busan_restrau_format.loc[i, 'RESERVABLE'] = details.get('reservable')
        busan_restrau_format.loc[i, 'BREAKFAST_YN'] = details.get('servesBreakfast')
        busan_restrau_format.loc[i, 'LUNCH_YN'] = details.get('servesLunch')
        busan_restrau_format.loc[i, 'DINNER_YN'] = details.get('servesDinner')
        busan_restrau_format.loc[i, 'BEER_YN'] = details.get('servesBeer')
        busan_restrau_format.loc[i, 'OUTDOOR_SEAT_YN'] = details.get('outdoorSeating')
        busan_restrau_format.loc[i, 'MENU_FOR_CHILDREN_YN'] = details.get('menuForChildren')
        busan_restrau_format.loc[i, 'RESTROOM_YN'] = details.get('restroom')
        busan_restrau_format.loc[i, 'PARKING_LOT_YN'] = format_multiple_options(details.get('parkingOptions'))
        busan_restrau_format.loc[i, 'PAYMENT_OPTIONS'] = format_multiple_options(details.get('paymentOptions'))
        busan_restrau_format.loc[i, 'REVIEW'] = format_review(details.get("reviews"))
        
    else:
        continue

{'candidates': [{'place_id': 'ChIJlY3wY1zsaDURLHegQBYnNig'}], 'status': 'OK'}
{'types': ['restaurant', 'food', 'point_of_interest', 'establishment'], 'formattedAddress': 'Busan, South Korea', 'rating': 4.3, 'userRatingCount': 37, 'displayName': {'text': '행복한빅마마삼겹', 'languageCode': 'ko'}, 'reservable': True, 'servesLunch': True, 'servesDinner': True, 'servesBeer': True, 'servesBrunch': False, 'reviews': [{'name': 'places/ChIJlY3wY1zsaDURLHegQBYnNig/reviews/ChdDSUhNMG9nS0VJQ0FnSURDcUota2x3RRAB', 'relativePublishTimeDescription': '4 years ago', 'rating': 4, 'text': {'text': "A reasonably delicious house. They mainly sell thinly sliced \u200b\u200bpork belly. The characteristics of this restaurant include light meat, numerous self-service side dishes, and the typical taste of thinly sliced \u200b\u200bpork belly. The taste is just delicious, and it's a decent restaurant that's cleaner than other thinly sliced \u200b\u200bpork belly restaurants that don't have anything special to say.", 'la

In [46]:
busan_restrau_format.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE,RESTAURANT_NAME_ENGLISH,ADDRESS_ENGLISH,...,BREAKFAST_YN,LUNCH_YN,DINNER_YN,BEER_YN,OUTDOOR_SEAT_YN,MENU_FOR_CHILDREN_YN,RESTROOM_YN,PARKING_LOT_YN,PAYMENT_OPTIONS,REVIEW
0,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),돼지갈비,051-852-0218,연제구,35.187,129.080,None,"Busan, South Korea",...,None,True,True,True,False,False,True,None,acceptsCreditCards,A reasonably delicious house. They mainly sell...
1,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",대구탕,051-867-7611,연제구,35.187,129.080,None,"Busan, South Korea",...,True,True,True,True,False,False,True,freeParkingLot,acceptsCreditCards,Not bad\nNice and fresh fish and soup~~\nGood\...
2,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)",뷔페,051-867-7600,연제구,35.186,129.081,Haeam Buffet Wedding Hall,"South Korea, Busan, Yeonje-gu, Jungang-daero, ...",...,None,None,None,None,None,None,None,None,None,Good\nIt is located near Exit 3 of Yeonsan Sub...
3,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),보쌈,051-853-8005,연제구,35.186,129.080,None,"South Korea, Seoul, Gwanak-gu, 신림동 1432-76",...,None,True,True,True,False,False,True,None,acceptsCreditCards,A house where slow eating is forced\n\nI visit...
4,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
busan_restrau_format.tail()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE,RESTAURANT_NAME_ENGLISH,ADDRESS_ENGLISH,...,BREAKFAST_YN,LUNCH_YN,DINNER_YN,BEER_YN,OUTDOOR_SEAT_YN,MENU_FOR_CHILDREN_YN,RESTROOM_YN,PARKING_LOT_YN,PAYMENT_OPTIONS,REVIEW
621,복어취급,금수복국 온천점,부산광역시 동래구 온천장로107번길 18 (온천동),복국,051-555-9912,부산광역시 동래구,35.220544,129.083349,None,"18 Oncheonjang-ro 107beon-gil, Dongnae-gu, Bus...",...,True,True,True,True,False,False,True,None,acceptsCreditCards,A restaurant specializing in puffer fish. Eat ...
622,한식,쌍둥이돼지국밥 본점,부산광역시 남구 유엔평화로 35-1 (대연동),돼지국밥,051-628-7021,부산광역시 남구,35.131770,129.092306,Ssangdungi (Twins) Dwaeji-gukbap Headstore,"35-1 UN pyeonghwa-ro, Nam-gu, Busan, South Korea",...,True,True,True,True,False,False,True,freeParkingLot,acceptsCreditCards,Their pork soup spot is regarded as one of the...
623,한식,오륙도 가원,부산광역시 남구 백운포로 14 (용호동),소 숯불구이,051-635-0707,부산광역시 남구,35.106842,129.111358,오륙도가원,"14 Baegunpo-ro, Nam-gu, Busan, South Korea",...,False,True,True,True,None,False,True,freeParkingLot,acceptsCreditCards,"An excellent barbecue place in a beautiful, cl..."
624,중국,가보자짬뽕,부산광역시 기장군 장안읍 해맞이로 425,중국요리,051-727-3911,부산광역시 기장군,35.329756,129.281918,None,"425 Haemaji-ro, Jangan-eup, Gijang, Busan, Sou...",...,None,True,True,True,None,None,True,None,acceptsCreditCards,It's a jjamppong restaurant.\nI visited with m...
625,한식,금정어가,부산광역시 두실로 17(구서동),장어구이,051-514-8216,부산광역시 금정구,35.260477,129.089353,None,"South Korea, Busan, Geumjeong-gu, 구서제2동 159-45",...,None,True,True,True,False,False,True,None,acceptsCreditCards,The wife was kind and the food was good.\nAll ...


In [48]:
busan_restrau_format.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   MENU_TYPE                626 non-null    object 
 1   RESTAURANT_NAME_KOREAN   626 non-null    object 
 2   ADDRESS_KOREAN           626 non-null    object 
 3   MENU_NAME                626 non-null    object 
 4   NATIONAL_PHONE_NUMBER    626 non-null    object 
 5   DISTRICT_NAME            626 non-null    object 
 6   LATITUDE                 626 non-null    float64
 7   LONGITUDE                626 non-null    float64
 8   RESTAURANT_NAME_ENGLISH  187 non-null    object 
 9   ADDRESS_ENGLISH          502 non-null    object 
 10  OPENING_HOURS            405 non-null    object 
 11  RATING                   477 non-null    float64
 12  RATING_COUNT             477 non-null    float64
 13  PRICE_LEVEL              228 non-null    object 
 14  TAKEOUT_YN               2

In [49]:
# busan_restrau_format.info()
# busan_restrau_format.to_csv("./datas/busan_restrau_20to24_witch_eng_data.csv", index=False, encoding="utf-8")

In [50]:
busan_restrau_dir = "/mnt/t7/dnn/llm_practicing/05_RAG_practicing/datas/busan_restrau_20to24.csv"
busan_restrau_df_new = pd.read_csv(busan_restrau_dir)

In [51]:
busan_restrau_format_new = busan_restrau_df_new[['BZCND_NM', 'RSTRNT_NM', 'LOCPLC_ALL_ADDR', 'MENU_NM', 'TEL_NO', 'GUGUN_NM', 'GID_LA', 'GID_LO']].copy()
busan_restrau_format_new.rename(columns={'BZCND_NM':'MENU_TYPE', 'RSTRNT_NM':'RESTAURANT_NAME_KOREAN', 'LOCPLC_ALL_ADDR': 'ADDRESS_KOREAN', 'MENU_NM':'MENU_NAME', 'TEL_NO':'NATIONAL_PHONE_NUMBER', 'GUGUN_NM':'DISTRICT_NAME', 'GID_LA':'LATITUDE', 'GID_LO':'LONGITUDE'},inplace = True)
busan_restrau_format_new.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE
0,식육(숯불구이),행복한빅마마,부산광역시 연제구 월드컵대로 151 (연산동),돼지갈비,051-852-0218,연제구,35.187,129.080
1,한식,해운대 신 대구탕,"부산광역시 연제구 월드컵대로 146 (연산동,1층)",대구탕,051-867-7611,연제구,35.187,129.080
2,뷔페식,해암뷔페,"부산광역시 연제구 중앙대로 1099 (연산동,,13)",뷔페,051-867-7600,연제구,35.186,129.081
3,한식,할매보쌈,부산광역시 연제구 월드컵대로145번길 11 (연산동),보쌈,051-853-8005,연제구,35.186,129.080
4,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075


In [55]:
# 짝수 인덱스의 행을 저장할 새로운 데이터프레임 생성 (빈 데이터프레임)
Nan_id_rows_df = pd.DataFrame(columns=busan_restrau_format_new.columns)

for i, row in busan_restrau_format_new.iterrows():
    time.sleep(0.1)
    restaurant_name = row["RESTAURANT_NAME_KOREAN"]
    location = f"{row['LATITUDE'],row['LONGITUDE']}" 
    place_id = get_place_id(restaurant_name, location)

    if not place_id:
        print("None!!!!!!!!!!!!!")
        Nan_id_rows_df = pd.concat([Nan_id_rows_df, row.to_frame().T], ignore_index=True)
        print(row)
    else:
        continue



{'candidates': [{'place_id': 'ChIJlY3wY1zsaDURLHegQBYnNig'}], 'status': 'OK'}
{'candidates': [{'place_id': 'ChIJoZqygvXCaDURvSWQbnbOlj4'}], 'status': 'OK'}
{'candidates': [{'place_id': 'ChIJdSpx0FqTaDUReIKe0GSiIIE'}], 'status': 'OK'}
{'candidates': [{'place_id': 'ChIJsy7N776ffDURlQMEmXx1dEQ'}], 'status': 'OK'}
{'candidates': [], 'status': 'ZERO_RESULTS'}
None!!!!!!!!!!!!!
{'candidates': [{'place_id': 'ChIJEWIypFqTaDUROSPOtaWo_OE'}], 'status': 'OK'}
{'candidates': [], 'status': 'ZERO_RESULTS'}
None!!!!!!!!!!!!!
{'candidates': [], 'status': 'ZERO_RESULTS'}
None!!!!!!!!!!!!!
{'candidates': [{'place_id': 'ChIJ_5XFnFqTaDURkuNb0HXmn68'}], 'status': 'OK'}
{'candidates': [], 'status': 'ZERO_RESULTS'}
None!!!!!!!!!!!!!
{'candidates': [{'place_id': 'ChIJGYtOomSkfDUR3jw4WhsP2dg'}], 'status': 'OK'}
{'candidates': [{'place_id': 'ChIJgbDRmVSTaDURYWnDbFZylXk'}, {'place_id': 'ChIJxwmfvZfraDURwXC7-LwxigI'}], 'status': 'OK'}
{'candidates': [{'place_id': 'ChIJPeVt7XzpaDUR4ztkxWylG2k'}], 'status': 'OK'}
{

In [56]:
Nan_id_rows_df.head()

,MENU_TYPE,RESTAURANT_NAME_KOREAN,ADDRESS_KOREAN,MENU_NAME,NATIONAL_PHONE_NUMBER,DISTRICT_NAME,LATITUDE,LONGITUDE
0,중국식,팔선각,부산광역시 연제구 법원로32번길 10 (거제동),중식,051-507-9555,연제구,35.191,129.075
1,탕류(보신용),토곡정,부산광역시 연제구 과정로 112-1 (연산동),삼계탕,051-755-5666,연제구,35.181,129.108
2,한식,토곡식육식당,부산광역시 연제구 토곡로 7 (연산동),소고기 모듬구이,051-751-9280,연제구,35.183,129.106
3,한식,청담동 대구뽈찜 아구찜,"부산광역시 연제구 월드컵대로120번길 12 (연산동,1층)",아구찜,051-866-8008,연제구,35.185,129.083
4,회집,원조메기탕,부산광역시 연제구 거제시장로 15 (거제동),메기탕,051-852-8085,연제구,35.181,129.072


In [58]:
# Nan_id_rows_df.to_csv("./datas/busan_restrau_20to24_none_id_data.csv", index=False, encoding="utf-8")

In [57]:
Nan_id_rows_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   MENU_TYPE               124 non-null    object
 1   RESTAURANT_NAME_KOREAN  124 non-null    object
 2   ADDRESS_KOREAN          124 non-null    object
 3   MENU_NAME               124 non-null    object
 4   NATIONAL_PHONE_NUMBER   124 non-null    object
 5   DISTRICT_NAME           124 non-null    object
 6   LATITUDE                124 non-null    object
 7   LONGITUDE               124 non-null    object
dtypes: object(8)
memory usage: 7.9+ KB
